In [1]:
import datetime
import tushare as ts
import api.storage as st
from api.ts_map import ts_mapping

In [2]:
storage = st.Storage()

In [3]:
with open('api/token/token.txt', "r") as f:  # 设置文件对象
    token = f.read()
pro = ts.pro_api(token)
# Get first 100 ts_codes
ts_codes = list(ts_mapping.keys())[:100]

In [4]:
df_rt = ts.get_realtime_quotes(ts_codes).set_index('code')
numeric_cols = [col for col in df_rt.columns if col not in {'name', 'date', 'time'}]
df_rt[numeric_cols] = df_rt[numeric_cols].replace('', '0').astype(np.float64)
df_rt.head()

,name,open,pre_close,price,high,low,bid,ask,volume,amount,...,a2_v,a2_p,a3_v,a3_p,a4_v,a4_p,a5_v,a5_p,date,time
code,,,,,,,,,,,,,,,,,,,,,
600805,悦达投资,5.00,5.02,4.92,5.03,4.91,4.92,4.93,4779752.0,23755747.0,...,184.0,4.94,242.0,4.95,147.0,4.96,260.0,4.97,2020-01-21,15:00:00
600809,山西汾酒,94.88,95.90,94.50,95.83,93.20,94.49,94.50,4031684.0,381596227.0,...,1.0,94.53,7.0,94.60,3.0,94.61,3.0,94.62,2020-01-21,15:00:00
002345,潮宏基,4.14,4.16,4.13,4.17,4.08,4.13,4.14,8754120.0,36023654.4,...,2420.0,4.15,3117.0,4.16,1063.0,4.17,951.0,4.18,2020-01-21,15:00:00
002694,顾地科技,3.19,3.20,3.12,3.24,3.11,3.12,3.13,8816700.0,27734622.0,...,565.0,3.14,890.0,3.15,478.0,3.16,377.0,3.17,2020-01-21,15:00:00
002696,百洋股份,6.08,6.09,5.95,6.09,5.94,5.95,5.96,8503805.0,51094864.3,...,854.0,5.97,396.0,5.98,725.0,5.99,1039.0,6.00,2020-01-21,15:00:00


In [5]:
df_basic = pro.daily_basic(ts_code='', trade_date=datetime.datetime.now().strftime('20200117'))
df_basic['code'] = [s[:-3] for s in df_basic['ts_code']]
df_basic = df_basic.set_index('code').loc[df_rt.index]
df_basic.head()

,ts_code,trade_date,close,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv
code,,,,,,,,,,,,,,,,,,
600805,600805.SH,20200117,4.95,0.4365,0.6421,0.74,51.8270,164.1394,0.6739,2.0197,1.6694,NaN,NaN,85089.4494,85027.5368,57802.8806,4.211928e+05,4.208863e+05
600809,600809.SH,20200117,96.69,0.3342,1.1389,0.47,57.4529,44.3787,11.6184,8.9819,7.2689,0.7757,0.7757,87152.8266,86584.8266,25408.5824,8.426807e+06,8.371887e+06
002345,002345.SZ,20200117,4.12,0.8204,1.6545,0.67,52.5063,89.0548,1.0643,1.1485,1.0661,NaN,NaN,90541.2707,87869.9215,43572.3832,3.730300e+05,3.620241e+05
002694,002694.SZ,20200117,3.20,1.5121,2.2350,0.64,NaN,NaN,1.9480,1.1482,1.2452,NaN,NaN,59719.6800,50878.8944,34423.0177,1.911030e+05,1.628125e+05
002696,002696.SZ,20200117,6.30,10.9522,10.9522,1.45,38.5214,NaN,1.1088,0.7024,0.7191,0.8980,0.8980,34938.6910,18290.7705,18290.7705,2.201138e+05,1.152319e+05


In [9]:
"""
@ File:     explode.py
@ Author:   pleiadesian
@ Datetime: 2020-01-16 20:51
@ Desc:     time share exploding detection
"""

import datetime
import tushare as ts
import api.ts_map

DEBUG = 0

OPEN_UPPER_LIMIT = 0.03
OPEN_LOWER_LIMIT = -0.02

RUSH_UPPER_LIMIT = 0.05
RUSH_LOWER_LIMIT = -0.03

MINUTE_LIMIT_THRESHOLD = 100  # 100 million volume of transaction

EXPLODE_RISE_RATIO_THRESHOLD = 0.0158


# TODO: scale to 30 codes
def detect_timeshare_explode(df_rt, df_basic, high_to_curr):
    volume_ma5 = 1
    
    tic1, tic2, toc = [pd.to_datetime(t) for t in ['9:30:00', '13:00:00', df_rt['time']]]
    minutes_lapse = (toc - tic1 - (toc >= tic2) * pd.Timedelta('1.5h')) / pd.Timedelta('1min')

    turnover_rate = df_rt['volume'] / df_basic['float_share'] / 100
    volume_ratio = df_rt['volume'] * 240 / volume_ma5 / minutes_lapse / 100

    today_open, pre_close = df_rt['open'], df_basic['close']
    open_ratio = (today_open - pre_close) / pre_close
    rise_ratio = (df_rt['price'] - pre_close) / pre_close

    rush_not_broken  = open_ratio    >= OPEN_LOWER_LIMIT 
    rush_not_broken &= open_ratio    <= OPEN_UPPER_LIMIT
    rush_not_broken &= df_rt['high'] <= RUSH_UPPER_LIMIT
    rush_not_broken &= df_rt['low']  >= RUSH_LOWER_LIMIT

    exploded  = df_rt['amount'] > MINUTE_LIMIT_THRESHOLD * 10000
    exploded &= rise_ratio >= EXPLODE_RISE_RATIO_THRESHOLD
    exploded &= df_rt['price'] > df_rt['high']
#     exploded &= high_to_curr   >= high_to_curr_threshold
    exploded &= turnover_rate >= 0.6
    exploded &= volume_ratio  > 0.6
    exploded &= rush_not_broken
    return exploded


res = detect_timeshare_explode(df_rt, df_basic, 29)
res

code
600805    False
600809    False
002345    False
002694    False
002696    False
          ...  
603583    False
603585    False
002889    False
002899    False
002905    False
Name: amount, Length: 100, dtype: bool

In [ ]:
# %load core/realtime/explode_vector.py
"""
@ File:     explode.py
@ Author:   pleiadesian
@ Datetime: 2020-01-16 20:51
@ Desc:     time share exploding detection
"""

import datetime
import tushare as ts
import api.ts_map

DEBUG = 0

OPEN_UPPER_LIMIT = 0.03
OPEN_LOWER_LIMIT = -0.02

RUSH_UPPER_LIMIT = 0.05
RUSH_LOWER_LIMIT = -0.03

MINUTE_LIMIT_THRESHOLD = 100  # 100 million volume of transaction

EXPLODE_RISE_RATIO_THRESHOLD = 0.0158


# TODO: scale to 30 codes
def detect_timeshare_explode(code, high_to_curr):
    assert(isinstance(code, list) is False)
    with open('../../api/token/token.txt', "r") as f:  # 设置文件对象
        token = f.read()
    pro = ts.pro_api(token)
    info = ts.get_realtime_quotes(code)
    info = info.values[0]
    not_get = True
    hist_data = None
    basic_infos = None
    while not_get:
        basic_infos = pro.daily_basic(ts_code=api.ts_map.ts_mapping[code],
                                      trade_date=datetime.datetime.now().strftime('20200117'))
        hist_data = ts.get_hist_data(code)
        if hist_data is None or basic_infos is None:
            continue
        not_get = False

    volume = float(info[8])
    volume_ma5 = float(hist_data['v_ma5'][0])
    today_open = float(info[1])
    pre_close = float(info[2])
    price = float(info[3])
    high = float(info[4])
    low = float(info[5])
    amount = float(info[9])

    time = info[31]
    time = datetime.datetime.strptime(time, "%H:%M:%S")
    if time <= datetime.datetime.strptime('11:30:00', "%H:%M:%S"):
        minutes_lapse = (time - datetime.datetime.strptime('9:30:00', "%H:%M:%S")).seconds / 60
    else:
        minutes_lapse = 120 + (time - datetime.datetime.strptime('13:00:00', "%H:%M:%S")).seconds / 60

    float_share = basic_infos['float_share'][0]
    turnover_rate = volume / float_share / 100
    volume_ratio = volume * 240 / volume_ma5 / minutes_lapse / 100

    open_ratio = (today_open - pre_close) / pre_close
    rise_ratio = (price - pre_close) / pre_close

    rush_not_broken = OPEN_LOWER_LIMIT <= open_ratio <= OPEN_UPPER_LIMIT
    rush_not_broken = rush_not_broken and high <= RUSH_UPPER_LIMIT
    rush_not_broken = rush_not_broken and low >= RUSH_LOWER_LIMIT

    exploded = amount / 10000 > MINUTE_LIMIT_THRESHOLD
    exploded = exploded and rise_ratio >= EXPLODE_RISE_RATIO_THRESHOLD
    exploded = exploded and price > high
    # exploded = exploded and high_to_curr >= high_to_curr_threshold
    exploded = exploded and turnover_rate >= 0.6
    exploded = exploded and volume_ratio > 0.6
    exploded = exploded and rush_not_broken
    return exploded


if __name__ == '__main__':
    exploded = detect_timeshare_explode('300496', 29)
    print(exploded)


